### Text Selection Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import sys
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Turkish
Lang Id = 2
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Text Selection/Result/3-Text Selection Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Arabic By Quran/Api Key/arabic-by-quran-firebase-adminsdk-l112z-6ce7c07252.json")  # Arabic By Quran
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
parent_folder_path = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Text Selection/Result/2-Text Selection Data Merge"

In [9]:
#df_word = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx")
df_word = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Text_Selection_Result.csv")
df_word = df_word.fillna("[]")
#df_word = df_word.iloc[0:1000,]
df_word.reset_index(inplace=True)
df_word

,index,word,frequency,twogram,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,0,bir,18835735,"['bir şey', 'bu bir', 'bir dakika', 'bir şeyle...","['bir şey yok', 'bir şey var', 'başka bir şey'...","['bir şey var mı', 'bir sorun mu var', 'gereke...","['başka bir şey var mı', 'sana bir şey sorabil...","['sana söylemem gereken bir şey var', 'bir iki...","['bir iki üç dört beş altı yedi', 'bana söylem...","['bir iki üç dört beş altı yedi sekiz', 'sekiz...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,1,bu,11062659,"['bu kadar', 'bu da', 'bu çok', 'bu bir', 'bu ...","['bu da ne', 'neden bu kadar', 'bu doğru değil...","['bu da ne demek', 'bu da ne böyle', 'bu iyi b...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['bu iyi bir şey değil mi', 'bence bu iyi bir ...","['bu sana bir şey ifade ediyor mu', 'evime var...","['bu sabah uyandın bütün o sevgi yok olmuş', '...",['bu gün bir kez daha biri bir yerlerde ağlaya...,['bu işlemi bu akşam başlatmak istiyorsak yapa...
2,2,ne,8025880,"['ne oldu', 'ne kadar', 'ne yapıyorsun', 'ne d...","['bu da ne', 'ne demek istiyorsun', 'ne işin v...","['burada ne işin var', 'bu da ne demek', 'en s...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['onu en son ne zaman gördün', 'bunun ne anlam...",['senin için ne kadar önemli olduğunu biliyoru...,['bunun senin için ne kadar zor olduğunu biliy...,['efendi luke sizi tekrar işler vaziyette görm...,['ne dediğimi duymadın mı evlat ben de ona eve...
3,3,ve,7766036,"['ve ben', 've bu', 've bir', 'sen ve', 've se...","['sen ve ben', 'bayanlar ve baylar', 've ben d...","['sadece sen ve ben', 'oğul ve kutsal ruh', 'b...","['baba oğul ve kutsal ruh', 'oğul ve kutsal ru...","['baba oğul ve kutsal ruh adına', 'evrenin her...","['bir ve iki ve üç ve dört', 've sen herkesin ...",['ve sen herkesin seni tanıdığı o yerde olmak'...,['ve sen herkesin seni tanıdığı o yerde olmak ...,['zamanımızı geri almak istiyorum ama şans ve ...
4,4,için,5484109,"['senin için', 'benim için', 'için bir', 'bunu...","['için özür dilerim', 'için teşekkür ederim', ...","['sizin için ne yapabilirim', 'senin için ne y...","['rahatsız ettiğim için özür dilerim', 'senin ...","['için yapabileceğim bir şey var mı', 'benim i...","['senin için yapabileceğim bir şey var mı', 's...",['sizin için yapabileceğim başka bir şey var m...,['buralara kadar gelebilmek için pek çok duvar...,['annem bana bazı şeyleri anlatmak için mutlak...
...,...,...,...,...,...,...,...,...,...,...,...,...
43045,43045,aştığını,871,[],[],[],[],['oldukça aştığını bilerek yüzümde bir gülümse...,['oldukça aştığını bilerek yüzümde bir gülümse...,[],[],[]
43046,43046,incitmiş,871,[],[],[],[],"['onu incitmiş o da öç almak', 'birisi onu inc...","['onu incitmiş o da öç almak istiyor', 'birisi...","['birisi onu incitmiş o da öç almak istiyor', ...",[],[]
43047,43047,kardeşisin,871,"['onun kardeşisin', 'kız kardeşisin']","['kardeşisin değil mi', 'onun kız kardeşisin']","['de onun kız kardeşisin', 'kız kardeşisin değ...",['sen de onun kız kardeşisin'],[],[],[],[],['kralın kardeşisin demek ona rağmen ordusuna ...
43048,43048,kaldırıyorsun,871,['kadar kaldırıyorsun'],[],[],[],[],[],[],[],[]


In [10]:
df_word.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43050 entries, 0 to 43049
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      43050 non-null  int64 
 1   word       43050 non-null  object
 2   frequency  43050 non-null  int64 
 3   twogram    43050 non-null  object
 4   threegram  43050 non-null  object
 5   fourgram   43050 non-null  object
 6   fivegram   43050 non-null  object
 7   sixgram    43050 non-null  object
 8   sevengram  43050 non-null  object
 9   eightgram  43050 non-null  object
 10  ninegram   43050 non-null  object
 11  tengram    43050 non-null  object
dtypes: int64(2), object(10)
memory usage: 3.9+ MB


In [11]:
#df_twogram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Twogram_Text_Selection_Result.xlsx")
df_twogram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Twogram_Text_Selection_Result.csv")
df_twogram = df_twogram.fillna("[]")
df_twogram = df_twogram.iloc[0:200000,]
df_twogram.reset_index(inplace=True)
df_twogram

,index,twogram,frequency,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,0,bir şey,859944,"['bir şey yok', 'bir şey var', 'başka bir şey'...","['bir şey var mı', 'gereken bir şey var', 'baş...","['başka bir şey var mı', 'sana bir şey sorabil...","['sana söylemem gereken bir şey var', 'söyleme...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,"['bu iyi bir şey mi kötü bir şey mi', 'daha ön...",['bana sadece içinde şeker olmayan bir şey ver...
1,1,değil mi,585879,"['öyle değil mi', 'biliyorsun değil mi', 'güze...","['de öyle değil mi', 'bir şey değil mi', 'çok ...","['sence de öyle değil mi', 'sizce de öyle deği...","['bu iyi bir şey değil mi', 'ne demek istediği...","['o kadar da kötü değildi değil mi', 'o kadar ...",['bir kadının şarkı söylemesi oldukça şaşırtıc...,['evimde bir kadının şarkı söylemesi oldukça ş...,['ama bugün yeni bir maç var galiba değil mi m...
2,2,ben de,377765,"['ben de seni', 'ben de öyle', 've ben de', 'e...","['ben de seni seviyorum', 'ben de öyle düşünmü...","['ben de seni gördüğüme sevindim', 'evet ben d...","['ben de en az senin kadar', 'ben de sana aynı...","['ben de sizi karı koca ilan ediyorum', 'ben d...",['siobhan intihar etti ben de onun yerine geçt...,['siz formları alırsınız ben de onları hazırla...,['ne dediğimi duymadın mı evlat ben de ona eve...
3,3,teşekkür ederim,370619,"['çok teşekkür ederim', 'için teşekkür ederim'...","['için çok teşekkür ederim', 'geldiğiniz için ...","['her şey için teşekkür ederim', 'zaman ayırdı...","['her şey için çok teşekkür ederim', 'beni kab...",['olmama izin verdiğin için tekrar teşekkür ed...,['gibi olmama izin verdiğin için tekrar teşekk...,['kendim gibi olmama izin verdiğin için tekrar...,['önce parfüm mağazasından alınmış video kaydı...
4,4,ne oldu,322758,"['sonra ne oldu', 'ne oldu sana', 'sana ne old...","['bil bakalım ne oldu', 'sana ne oldu böyle', ...","['az önce ne oldu öyle', 'sonra ne oldu biliyo...","['ne oldu bir sorun mu var', 'az önce ne oldu ...",['gelecek hakkındaki bilgileri reddetme konuşm...,['insanlık nereye gidiyoruz ve niçin soruların...,['ya insanlık nereye gidiyoruz ve niçin sorula...,['burnuna ne oldu gittes yoksa kapı yerine bac...
...,...,...,...,...,...,...,...,...,...,...,...
179456,179456,kaşık daha,52,['bir kaşık daha'],[],[],[],[],[],[],[]
179457,179457,farklı görünüyordu,52,[],[],[],[],[],['artık daha büyük olduğu için kasaba farklı g...,[],[]
179458,179458,insanlığı yok,52,[],[],[],['bütün insanlığı yok edecek güce sahip'],[],[],[],[]
179459,179459,olmadan gerçek,52,['olmadan gerçek bir'],[],[],[],[],[],[],[]


In [12]:
#df_threegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Threegram_Text_Selection_Result.xlsx")
df_threegram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Threegram_Text_Selection_Result.csv")
df_threegram = df_threegram.fillna("[]")
df_threegram = df_threegram.iloc[0:200000,]
df_threegram.reset_index(inplace=True)
df_threegram

,index,threegram,frequency,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,0,bir şey yok,113165,"['diye bir şey yok', 'başka bir şey yok', 'bir...","['biz diye bir şey yok', 'başka bir şey yok mu...","['bir şey yok bir şey yok', 'yok bir şey yok b...","['söylemek istediğin başka bir şey yok mu', 'b...",['birbirimiz olmadan ikimizin de yapacağı bir ...,['ölmeden önce de aşıktım ve yapabileceğim bir...,['o ölmeden önce de aşıktım ve yapabileceğim b...
1,1,bir şey var,110455,"['bir şey var mı', 'gereken bir şey var', 'baş...","['başka bir şey var mı', 'yapabileceğim bir şe...","['sana söylemem gereken bir şey var', 'söyleme...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,['ev sahibi babamız için bende özel bir şey va...,['kaosun içinde düzen için mücadele eden tek b...
2,2,bu da ne,89463,"['bu da ne demek', 'bu da ne böyle', 'bu da ne...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['bu da ne demek oluyor şimdi', 'bu da ne deme...","['bu da ne biçim bir soru böyle', 'ama bu da n...","['alınmayın ama bu da ne demek oluyor şimdi', ...",[],['bu da ne olursa olsun oynamaya karar verdiği...
3,3,her şey yolunda,75968,"['her şey yolunda mı', 'her şey yolunda gidece...","['orada her şey yolunda mı', 'burada her şey y...","['her şey yolunda her şey yolunda', 'merak etm...",['helikopterler bizi koruyacak her şey yolunda...,"['her şey yolunda giderse çok yakında bir ev',...",['her şey yolunda giderse çok yakında bir ev a...,['dostlar acil bir durum yaşadık ama şimdi her...
4,4,başka bir şey,75193,"['başka bir şey var', 'başka bir şey yok', 'ba...","['başka bir şey var mı', 'başka bir şey daha v...","['istediğin başka bir şey var mı', 'yapabilece...","['için yapabileceğim başka bir şey var mı', 'b...",['sizin için yapabileceğim başka bir şey var m...,['kılıcını ve onun kılıcını düşün başka bir şe...,['sabırsızlıkla beklediğim zevkle yapacağım ba...
...,...,...,...,...,...,...,...,...,...,...
199995,199995,şu tarafa bir,41,[],[],['tamam şimdi şu tarafa bir göz'],[],[],[],[]
199996,199996,ve tüm ışıklar,41,[],[],[],[],[],[],['şu an itibariyle tüm sistemler hazır ve tüm ...
199997,199997,levha gördün mü,41,['bir levha gördün mü'],['yazan bir levha gördün mü'],"['deposu yazan bir levha gördün mü', 'deposu d...","['zenci deposu yazan bir levha gördün mü', 'ze...",['ölü zenci deposu yazan bir levha gördün mü'],[],[]
199998,199998,sen bir yok,41,['ama sen bir yok'],[],['yanlış anlama ama sen bir yok'],[],[],[],[]


In [13]:
#df_fourgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.xlsx")
df_fourgram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.csv")
df_fourgram = df_fourgram.fillna("[]")
df_fourgram = df_fourgram.iloc[0:200000,]
df_fourgram.reset_index(inplace=True)
df_fourgram

<ipython-input-13-aba3a0fdb7c1>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fourgram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.csv")


,index,fourgram,frequency,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,0,bir şey var mı,41773,"['başka bir şey var mı', 'yapabileceğim bir şe...","['söylemek istediğin bir şey var mı', 'için ya...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,['sizin için yapabileceğim başka bir şey var m...,['tanrı aşkına çalışmak için yaşamak diye bir ...
1,1,her şey yolunda mı,31126,"['orada her şey yolunda mı', 'burada her şey y...","['her şey yolunda mı diye bakmak', 'her şey yo...","['her şey yolunda mı diye bakmak istedim', 'he...","['git de her şey yolunda mı bir bak', 'hadi gi...",['hadi git de her şey yolunda mı bir bak'],['yukarı gelip her şey yolunda mı diye kontrol...
2,2,burada ne işin var,21993,"['senin burada ne işin var', 'burada ne işin v...","['o zaman burada ne işin var', 'bu saatte bura...","['gecenin bu saatinde burada ne işin var', 'on...",['o kadar paran varsa burada ne işin var'],[],[]
3,3,bir sorun mu var,21423,"['bir sorun mu var efendim', 'ilgili bir sorun...","['bir sorun mu var memur bey', 'ile ilgili bir...","['yerçekimi ile ilgili bir sorun mu var', 'dem...",['küresel yerçekimi ile ilgili bir sorun mu va...,['neden demin geldiğimde bir sorun mu var diye...,['merak ediyorum da acaba bende bir sorun mu v...
4,4,ben de seni seviyorum,17338,"['ben de seni seviyorum baba', 'ben de seni se...","['ben de seni seviyorum biliyorsun değil', 'be...","['ben de seni seviyorum biliyorsun değil mi', ...","['bu çok güzel çünkü ben de seni seviyorum', '...",[],['ben de seni seviyorum tatlım hem de her şeyd...
...,...,...,...,...,...,...,...,...,...
199995,199995,set ve maç bay,19,"['set ve maç bay haines', 'oyun set ve maç bay']",['oyun set ve maç bay haines'],[],[],[],[]
199996,199996,ya da düzgün bir,19,[],[],[],[],[],['ödevimi yapmamı ya da düzgün bir saatte evde...
199997,199997,zafer ve onur arayışı,19,['zafer ve onur arayışı içinde'],[],[],[],[],[]
199998,199998,ederim ki sana bir,19,"['ederim ki sana bir zarar', 'yemin ederim ki ...","['adına yemin ederim ki sana bir', 'ederim ki ...","['yemin ederim ki sana bir zarar vermez', 'adı...",['adına yemin ederim ki sana bir zarar vermez'...,['tanrı adına yemin ederim ki sana bir zarar v...,[]


In [14]:
#df_fivegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.xlsx")
df_fivegram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.csv")
df_fivegram = df_fivegram.fillna("[]")
df_fivegram = df_fivegram.iloc[0:200000,]
df_fivegram.reset_index(inplace=True)
df_fivegram

,index,fivegram,frequency,sixgram,sevengram,eightgram,ninegram,tengram
0,0,başka bir şey var mı,14104,"['istediğin başka bir şey var mı', 'yapabilece...","['için yapabileceğim başka bir şey var mı', 'b...",['sizin için yapabileceğim başka bir şey var m...,['sizin için yapabileceğim başka bir şey var m...,['affedersiniz bayan sizin için yapabileceğim ...
1,1,bu da ne demek oluyor,10205,"['bu da ne demek oluyor şimdi', 'bu da ne deme...","['ama bu da ne demek oluyor şimdi', 'alınmayın...",['alınmayın ama bu da ne demek oluyor şimdi'],[],['bu da ne demek oluyor yargılamak gibi bir şe...
2,2,o kadar da kötü değil,7012,"['bu o kadar da kötü değil', 'o kadar da kötü ...","['o kadar da kötü değil değil mi', 'hadi ama o...","['bu o kadar da kötü değil değil mi', 'ama o k...",['hadi ama pascal burası o kadar da kötü değil...,['hem prenses olmak o kadar da kötü değil deği...
3,3,sence de öyle değil mi,6305,"['görünüyor sence de öyle değil mi', 'var senc...","['iyi gidiyor sence de öyle değil mi', 'zamand...","['hoş bir vadi sence de öyle değil mi', 'çok d...","['burası hoş bir vadi sence de öyle değil mi',...",['gelecek geçmişten çok daha ilginç sence de ö...
4,4,sana bir şey sorabilir miyim,6224,"['hey sana bir şey sorabilir miyim', 'baba san...",['kimseye söylemezsen sana bir şey sorabilir m...,[],[],[]
...,...,...,...,...,...,...,...,...
87094,87094,beni çaldın ben de kendimi,19,"['beni çaldın ben de kendimi geri', 'sen beni ...","['beni çaldın ben de kendimi geri almaya', 'se...","['sen beni çaldın ben de kendimi geri almaya',...",['sen beni çaldın ben de kendimi geri almaya g...,[]
87095,87095,madem bir melek gibi çalıyorum,19,['madem bir melek gibi çalıyorum neden'],['madem bir melek gibi çalıyorum neden beni'],['madem bir melek gibi çalıyorum neden beni di...,[],[]
87096,87096,ile matrix arasında seçim yapmam,19,"['ile matrix arasında seçim yapmam gerekse', '...","['bu yaşam ile matrix arasında seçim yapmam', ...",['bu yaşam ile matrix arasında seçim yapmam ge...,[],[]
87097,87097,beklemek için nasıl bir yer,19,['beklemek için nasıl bir yer burası'],[],[],[],[]


In [15]:
#df_sixgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.xlsx")
df_sixgram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.csv")
df_sixgram = df_sixgram.fillna("[]")
df_sixgram = df_sixgram.iloc[0:200000,]
df_sixgram.reset_index(inplace=True)
df_sixgram

,index,sixgram,frequency,sevengram,eightgram,ninegram,tengram
0,0,sana söylemem gereken bir şey var,3646,"['ama sana söylemem gereken bir şey var', 'önc...","['benim de sana söylemem gereken bir şey var',...",['sana söylemem gereken bir şey var ama nasıl ...,['sana söylemem gereken bir şey var ama nasıl ...
1,1,bir iki üç dört beş altı,2025,"['bir iki üç dört beş altı yedi', 've bir iki ...","['bir iki üç dört beş altı yedi sekiz', 've bi...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
2,2,söylemek istediğin bir şey var mı,1794,"['bana söylemek istediğin bir şey var mı', 'bi...",['önce bana söylemek istediğin bir şey var mı'...,['çıkmadan önce bana söylemek istediğin bir şe...,['arabadan çıkmadan önce bana söylemek istediğ...
3,3,daha iyi bir fikrin var mı,1792,"['senin daha iyi bir fikrin var mı', 'peki dah...","['peki daha iyi bir fikrin var mı şerif', 'lan...",[],['benim dahil olmadığım daha iyi bir fikrin va...
4,4,bunun iyi bir fikir olduğunu sanmıyorum,1743,['ama bunun iyi bir fikir olduğunu sanmıyorum'...,['hayır hayır bunun iyi bir fikir olduğunu san...,[],[]
...,...,...,...,...,...,...,...
199995,199995,iş yapmak için yeterli bilgiye sahip,10,"['bir iş yapmak için yeterli bilgiye sahip', '...",['bir iş yapmak için yeterli bilgiye sahip olm...,[],[]
199996,199996,söylediğin gibi çok cesurca bir hareketti,10,['getirmek söylediğin gibi çok cesurca bir har...,['oraya getirmek söylediğin gibi çok cesurca b...,['babamı oraya getirmek söylediğin gibi çok ce...,['gidip babamı oraya getirmek söylediğin gibi ...
199997,199997,kutunun içinde hafif bir negatif basınç,10,['kutunun içinde hafif bir negatif basınç var'],[],[],[]
199998,199998,gidebilecek yaşa geldiğinde hiçbir havası kalm...,10,['gidebilecek yaşa geldiğinde hiçbir havası ka...,['ona gidebilecek yaşa geldiğinde hiçbir havas...,[],[]


In [16]:
#df_sevengram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.xlsx")
df_sevengram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.csv")
df_sevengram = df_sevengram.fillna("[]")
df_sevengram = df_sevengram.iloc[0:200000,]
df_sevengram.reset_index(inplace=True)
df_sevengram

,index,sevengram,frequency,eightgram,ninegram,tengram
0,0,bir iki üç dört beş altı yedi,1455,"['bir iki üç dört beş altı yedi sekiz', 've bi...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,1,iki üç dört beş altı yedi sekiz,1080,"['bir iki üç dört beş altı yedi sekiz', 'iki ü...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
2,2,bana söylemek istediğin bir şey var mı,752,['önce bana söylemek istediğin bir şey var mı'...,['çıkmadan önce bana söylemek istediğin bir şe...,['arabadan çıkmadan önce bana söylemek istediğ...
3,3,yedi altı beş dört üç iki bir,596,"['sekiz yedi altı beş dört üç iki bir', 'yedi ...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
4,4,sekiz yedi altı beş dört üç iki,551,"['dokuz sekiz yedi altı beş dört üç iki', 'sek...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
...,...,...,...,...,...,...
102080,102080,akşam üçüncü ve muhtemelen son olarak yerde,9,['bu akşam üçüncü ve muhtemelen son olarak yer...,['atom bu akşam üçüncü ve muhtemelen son olara...,[]
102081,102081,herifi durdurmak için sadece bir saat kırk,9,['herifi durdurmak için sadece bir saat kırk d...,['bu herifi durdurmak için sadece bir saat kır...,['bu herifi durdurmak için sadece bir saat kır...
102082,102082,görücü usulü evlenmek daha çok işe yarıyor,9,['bazen görücü usulü evlenmek daha çok işe yar...,[],[]
102083,102083,konuda çok yanlış olduğundan kesinlikle eminim...,9,['bu konuda çok yanlış olduğundan kesinlikle e...,[],[]


In [17]:
#df_eightgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.xlsx")
df_eightgram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.csv")
df_eightgram = df_eightgram.fillna("[]")
df_eightgram = df_eightgram.iloc[0:200000,]
df_eightgram.reset_index(inplace=True)
df_eightgram

,index,eightgram,frequency,ninegram,tengram
0,0,bir iki üç dört beş altı yedi sekiz,1060,"['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,1,dokuz sekiz yedi altı beş dört üç iki,513,"['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
2,2,sekiz yedi altı beş dört üç iki bir,497,"['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
3,3,iki üç dört beş altı yedi sekiz dokuz,411,"['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
4,4,üç dört beş altı yedi sekiz dokuz on,329,"['iki üç dört beş altı yedi sekiz dokuz on', '...",['bir iki üç dört beş altı yedi sekiz dokuz on...
...,...,...,...,...,...
71092,71092,bu bok en uzun süre ne kadar etki,7,[],['bu bok en uzun süre ne kadar etki gösterdi s...
71093,71093,şu anda telefona gelemez ama beni bıraktığınız...,7,[],['joshua şu anda telefona gelemez ama beni bır...
71094,71094,istediğim elini içeri sokman ve kabloyu nazikç...,7,[],['senden istediğim elini içeri sokman ve kablo...
71095,71095,insan diğer bir insanın işine çomak sokmak zor...,7,[],['neden bir insan diğer bir insanın işine çoma...


In [18]:
#df_ninegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.xlsx")
df_ninegram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.csv")
df_ninegram = df_ninegram.fillna("[]")
df_ninegram = df_ninegram.iloc[0:200000,]
df_ninegram.reset_index(inplace=True)
df_ninegram

,index,ninegram,frequency,tengram
0,0,dokuz sekiz yedi altı beş dört üç iki bir,463,['on dokuz sekiz yedi altı beş dört üç iki bir...
1,1,bir iki üç dört beş altı yedi sekiz dokuz,405,['bir iki üç dört beş altı yedi sekiz dokuz on...
2,2,iki üç dört beş altı yedi sekiz dokuz on,287,['bir iki üç dört beş altı yedi sekiz dokuz on']
3,3,on dokuz sekiz yedi altı beş dört üç iki,277,['on dokuz sekiz yedi altı beş dört üç iki bir']
4,4,i t a r i s y e n,128,['g i t a r i s y e n']
...,...,...,...,...
20237,20237,sevgili kızım aşık bir kadın için bile mantıks...,7,['sevgili kızım aşık bir kadın için bile mantı...
20238,20238,çene çalmayı bir yana bırakıp bu işi gerçekten...,7,['çene çalmayı bir yana bırakıp bu işi gerçekt...
20239,20239,merak etme böyle bir şey yapmamı istemediğinde...,7,['merak etme böyle bir şey yapmamı istemediğin...
20240,20240,sabah dokuz gibi geçerken uğrarım bu yüzden ön...,7,['sabah dokuz gibi geçerken uğrarım bu yüzden ...


#### Sending Data To Firestore

##### Photo Data

In [19]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Text Data

##### Word

In [10]:
df_word["twogram"] = df_word["twogram"].apply(eval)
df_word["threegram"] = df_word["threegram"].apply(eval)
df_word["fourgram"] = df_word["fourgram"].apply(eval)
df_word["fivegram"] = df_word["fivegram"].apply(eval)
df_word["sixgram"] = df_word["sixgram"].apply(eval)
df_word["sevengram"] = df_word["sevengram"].apply(eval)
df_word["eightgram"] = df_word["eightgram"].apply(eval)
df_word["ninegram"] = df_word["ninegram"].apply(eval)
df_word["tengram"] = df_word["tengram"].apply(eval)

In [11]:
index = df_word["index"].to_list()
word = df_word["word"].to_list()
frequency_list = df_word["frequency"].to_list()
twogram_list = df_word["twogram"].to_list()
threegram_list = df_word["threegram"].to_list()
fourgram_list = df_word["fourgram"].to_list()
fivegram_list = df_word["fivegram"].to_list()
sixgram_list = df_word["sixgram"].to_list()
sevengram_list = df_word["sevengram"].to_list()
eightgram_list = df_word["eightgram"].to_list()
ninegram_list = df_word["ninegram"].to_list()
tengram_list = df_word["tengram"].to_list()

In [12]:
firestore_word_data = zip(index,word,frequency_list,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [13]:
try:
    for index,word,frequency_list,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_word_data:
        dict_var = {f"{index}":{f"{word}":{"frequency":frequency_list,"twogram":twogram_list,"threegram":threegram_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
        sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list}}}
        fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"word{index}").set(dict_var)
except:
    print("Oops!", index, sys.exc_info()[0], "occurred.")    

##### Update (For Third Work)

In [14]:
index = 20
#update_word = "açıkçası"

In [15]:
dict_update = {f"{index}.other":["sentence",{"eş anlam":"yeter"}]}

In [16]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"word{index}").update(dict_update)

update_time {
  seconds: 1672416213
  nanos: 23914000
}

##### Twogram

In [12]:
df_twogram

,index,twogram,frequency,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,0,bir şey,859944,"['bir şey yok', 'bir şey var', 'başka bir şey'...","['bir şey var mı', 'gereken bir şey var', 'baş...","['başka bir şey var mı', 'sana bir şey sorabil...","['sana söylemem gereken bir şey var', 'söyleme...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,"['bu iyi bir şey mi kötü bir şey mi', 'daha ön...",['bana sadece içinde şeker olmayan bir şey ver...
1,1,değil mi,585879,"['öyle değil mi', 'biliyorsun değil mi', 'güze...","['de öyle değil mi', 'bir şey değil mi', 'çok ...","['sence de öyle değil mi', 'sizce de öyle deği...","['bu iyi bir şey değil mi', 'ne demek istediği...","['o kadar da kötü değildi değil mi', 'o kadar ...",['bir kadının şarkı söylemesi oldukça şaşırtıc...,['evimde bir kadının şarkı söylemesi oldukça ş...,['ama bugün yeni bir maç var galiba değil mi m...
2,2,ben de,377765,"['ben de seni', 'ben de öyle', 've ben de', 'e...","['ben de seni seviyorum', 'ben de öyle düşünmü...","['ben de seni gördüğüme sevindim', 'evet ben d...","['ben de en az senin kadar', 'ben de sana aynı...","['ben de sizi karı koca ilan ediyorum', 'ben d...",['siobhan intihar etti ben de onun yerine geçt...,['siz formları alırsınız ben de onları hazırla...,['ne dediğimi duymadın mı evlat ben de ona eve...
3,3,teşekkür ederim,370619,"['çok teşekkür ederim', 'için teşekkür ederim'...","['için çok teşekkür ederim', 'geldiğiniz için ...","['her şey için teşekkür ederim', 'zaman ayırdı...","['her şey için çok teşekkür ederim', 'beni kab...",['olmama izin verdiğin için tekrar teşekkür ed...,['gibi olmama izin verdiğin için tekrar teşekk...,['kendim gibi olmama izin verdiğin için tekrar...,['önce parfüm mağazasından alınmış video kaydı...
4,4,ne oldu,322758,"['sonra ne oldu', 'ne oldu sana', 'sana ne old...","['bil bakalım ne oldu', 'sana ne oldu böyle', ...","['az önce ne oldu öyle', 'sonra ne oldu biliyo...","['ne oldu bir sorun mu var', 'az önce ne oldu ...",['gelecek hakkındaki bilgileri reddetme konuşm...,['insanlık nereye gidiyoruz ve niçin soruların...,['ya insanlık nereye gidiyoruz ve niçin sorula...,['burnuna ne oldu gittes yoksa kapı yerine bac...
...,...,...,...,...,...,...,...,...,...,...,...
179456,179456,kaşık daha,52,['bir kaşık daha'],[],[],[],[],[],[],[]
179457,179457,farklı görünüyordu,52,[],[],[],[],[],['artık daha büyük olduğu için kasaba farklı g...,[],[]
179458,179458,insanlığı yok,52,[],[],[],['bütün insanlığı yok edecek güce sahip'],[],[],[],[]
179459,179459,olmadan gerçek,52,['olmadan gerçek bir'],[],[],[],[],[],[],[]


In [13]:
df_twogram["threegram"] = df_twogram["threegram"].apply(eval)
df_twogram["fourgram"] = df_twogram["fourgram"].apply(eval)
df_twogram["fivegram"] = df_twogram["fivegram"].apply(eval)
df_twogram["sixgram"] = df_twogram["sixgram"].apply(eval)
df_twogram["sevengram"] = df_twogram["sevengram"].apply(eval)
df_twogram["eightgram"] = df_twogram["eightgram"].apply(eval)
df_twogram["ninegram"] = df_twogram["ninegram"].apply(eval)
df_twogram["tengram"] = df_twogram["tengram"].apply(eval)

In [14]:
#twogram = df_twogram["twogram"].values.tolist()

In [15]:
index = df_twogram["index"].to_list()
twogram = df_twogram["twogram"].to_list()
frequency_list = df_twogram["frequency"].to_list()
threegram_list = df_twogram["threegram"].to_list()
fourgram_list = df_twogram["fourgram"].to_list()
fivegram_list = df_twogram["fivegram"].to_list()
sixgram_list = df_twogram["sixgram"].to_list()
sevengram_list = df_twogram["sevengram"].to_list()
eightgram_list = df_twogram["eightgram"].to_list()
ninegram_list = df_twogram["ninegram"].to_list()
tengram_list = df_twogram["tengram"].to_list()

In [16]:
firestore_twogram_data = zip(index,twogram,frequency_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [18]:
try:
    for index,twogram,frequency_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_twogram_data:
        dict_var = {f"{index}":{f"{twogram}":{"frequency":frequency_list,"threegram":threegram_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
        sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list}}}
        fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("twogram_data").document(f"twogram{index}").set(dict_var)
except:
    print("Oops!", index, sys.exc_info()[0], "occurred.")

##### Threegram

In [14]:
df_threegram

,index,threegram,frequency,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,0,bir şey yok,113165,"['diye bir şey yok', 'başka bir şey yok', 'bir...","['biz diye bir şey yok', 'başka bir şey yok mu...","['bir şey yok bir şey yok', 'yok bir şey yok b...","['söylemek istediğin başka bir şey yok mu', 'b...",['birbirimiz olmadan ikimizin de yapacağı bir ...,['ölmeden önce de aşıktım ve yapabileceğim bir...,['o ölmeden önce de aşıktım ve yapabileceğim b...
1,1,bir şey var,110455,"['bir şey var mı', 'gereken bir şey var', 'baş...","['başka bir şey var mı', 'yapabileceğim bir şe...","['sana söylemem gereken bir şey var', 'söyleme...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,['ev sahibi babamız için bende özel bir şey va...,['kaosun içinde düzen için mücadele eden tek b...
2,2,bu da ne,89463,"['bu da ne demek', 'bu da ne böyle', 'bu da ne...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['bu da ne demek oluyor şimdi', 'bu da ne deme...","['bu da ne biçim bir soru böyle', 'ama bu da n...","['alınmayın ama bu da ne demek oluyor şimdi', ...",[],['bu da ne olursa olsun oynamaya karar verdiği...
3,3,her şey yolunda,75968,"['her şey yolunda mı', 'her şey yolunda gidece...","['orada her şey yolunda mı', 'burada her şey y...","['her şey yolunda her şey yolunda', 'merak etm...",['helikopterler bizi koruyacak her şey yolunda...,"['her şey yolunda giderse çok yakında bir ev',...",['her şey yolunda giderse çok yakında bir ev a...,['dostlar acil bir durum yaşadık ama şimdi her...
4,4,başka bir şey,75193,"['başka bir şey var', 'başka bir şey yok', 'ba...","['başka bir şey var mı', 'başka bir şey daha v...","['istediğin başka bir şey var mı', 'yapabilece...","['için yapabileceğim başka bir şey var mı', 'b...",['sizin için yapabileceğim başka bir şey var m...,['kılıcını ve onun kılıcını düşün başka bir şe...,['sabırsızlıkla beklediğim zevkle yapacağım ba...
...,...,...,...,...,...,...,...,...,...,...
213878,213878,götürmek için hazır,38,['götürmek için hazır bekliyor'],[],[],[],[],[],[]
213879,213879,hafta üç haftadır,38,['üç hafta üç haftadır'],[],['ki dinle üç hafta üç haftadır'],['dedim ki dinle üç hafta üç haftadır'],[],[],[]
213880,213880,olan birkaç şey,38,['olan birkaç şey var'],[],[],[],[],[],['benim de bildiğim dikkat edilmesi kesin olan...
213881,213881,da size anlatayım,38,"['durun da size anlatayım', 'da size anlatayım...","['durun da size anlatayım ama', 'da size anlat...","['da size anlatayım ama lütfen aramızda', 'dur...",['da size anlatayım ama lütfen aramızda kalsın...,['durun da size anlatayım ama lütfen aramızda ...,[],[]


In [15]:
df_threegram["fourgram"] = df_threegram["fourgram"].apply(eval)
df_threegram["fivegram"] = df_threegram["fivegram"].apply(eval)
df_threegram["sixgram"] = df_threegram["sixgram"].apply(eval)
df_threegram["sevengram"] = df_threegram["sevengram"].apply(eval)
df_threegram["eightgram"] = df_threegram["eightgram"].apply(eval)
df_threegram["ninegram"] = df_threegram["ninegram"].apply(eval)
df_threegram["tengram"] = df_threegram["tengram"].apply(eval)

In [16]:
index = df_threegram["index"].to_list()
threegram = df_threegram["threegram"].to_list()
frequency_list = df_threegram["frequency"].to_list()
fourgram_list = df_threegram["fourgram"].to_list()
fivegram_list = df_threegram["fivegram"].to_list()
sixgram_list = df_threegram["sixgram"].to_list()
sevengram_list = df_threegram["sevengram"].to_list()
eightgram_list = df_threegram["eightgram"].to_list()
ninegram_list = df_threegram["ninegram"].to_list()
tengram_list = df_threegram["tengram"].to_list()

In [17]:
firestore_threegram_data = zip(index,threegram,frequency_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [18]:
try:
    for index,threegram,frequency_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_threegram_data:
        dict_var = {f"{index}":{f"{threegram}":{"frequency":frequency_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
        sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list}}}
        fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("threegram_data").document(f"threegram{index}").set(dict_var)
except:
    print("Oops!", index, sys.exc_info()[0], "occurred.")       

##### Fourgram

In [19]:
df_fourgram

,index,fourgram,frequency,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,0,bir şey var mı,41773,"['başka bir şey var mı', 'yapabileceğim bir şe...","['söylemek istediğin bir şey var mı', 'için ya...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,['sizin için yapabileceğim başka bir şey var m...,['tanrı aşkına çalışmak için yaşamak diye bir ...
1,1,her şey yolunda mı,31126,"['orada her şey yolunda mı', 'burada her şey y...","['her şey yolunda mı diye bakmak', 'her şey yo...","['her şey yolunda mı diye bakmak istedim', 'he...","['git de her şey yolunda mı bir bak', 'hadi gi...",['hadi git de her şey yolunda mı bir bak'],['yukarı gelip her şey yolunda mı diye kontrol...
2,2,burada ne işin var,21993,"['senin burada ne işin var', 'burada ne işin v...","['o zaman burada ne işin var', 'bu saatte bura...","['gecenin bu saatinde burada ne işin var', 'on...",['o kadar paran varsa burada ne işin var'],[],[]
3,3,bir sorun mu var,21423,"['bir sorun mu var efendim', 'ilgili bir sorun...","['bir sorun mu var memur bey', 'ile ilgili bir...","['yerçekimi ile ilgili bir sorun mu var', 'dem...",['küresel yerçekimi ile ilgili bir sorun mu va...,['neden demin geldiğimde bir sorun mu var diye...,['merak ediyorum da acaba bende bir sorun mu v...
4,4,ben de seni seviyorum,17338,"['ben de seni seviyorum baba', 'ben de seni se...","['ben de seni seviyorum biliyorsun değil', 'be...","['ben de seni seviyorum biliyorsun değil mi', ...","['bu çok güzel çünkü ben de seni seviyorum', '...",[],['ben de seni seviyorum tatlım hem de her şeyd...
...,...,...,...,...,...,...,...,...,...
199995,199995,set ve maç bay,19,"['set ve maç bay haines', 'oyun set ve maç bay']",['oyun set ve maç bay haines'],[],[],[],[]
199996,199996,ya da düzgün bir,19,[],[],[],[],[],['ödevimi yapmamı ya da düzgün bir saatte evde...
199997,199997,zafer ve onur arayışı,19,['zafer ve onur arayışı içinde'],[],[],[],[],[]
199998,199998,ederim ki sana bir,19,"['ederim ki sana bir zarar', 'yemin ederim ki ...","['adına yemin ederim ki sana bir', 'ederim ki ...","['yemin ederim ki sana bir zarar vermez', 'adı...",['adına yemin ederim ki sana bir zarar vermez'...,['tanrı adına yemin ederim ki sana bir zarar v...,[]


In [20]:
df_fourgram["fivegram"] = df_fourgram["fivegram"].apply(eval)
df_fourgram["sixgram"] = df_fourgram["sixgram"].apply(eval)
df_fourgram["sevengram"] = df_fourgram["sevengram"].apply(eval)
df_fourgram["eightgram"] = df_fourgram["eightgram"].apply(eval)
df_fourgram["ninegram"] = df_fourgram["ninegram"].apply(eval)
df_fourgram["tengram"] = df_fourgram["tengram"].apply(eval)

In [21]:
index = df_fourgram["index"].to_list()
fourgram = df_fourgram["fourgram"].to_list()
frequency_list = df_fourgram["frequency"].to_list()
fivegram_list = df_fourgram["fivegram"].to_list()
sixgram_list = df_fourgram["sixgram"].to_list()
sevengram_list = df_fourgram["sevengram"].to_list()
eightgram_list = df_fourgram["eightgram"].to_list()
ninegram_list = df_fourgram["ninegram"].to_list()
tengram_list = df_fourgram["tengram"].to_list()

In [22]:
firestore_fourgram_data = zip(index,fourgram,frequency_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [23]:
try:
    for index,fourgram,frequency_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_fourgram_data:
        dict_var = {f"{index}":{f"{fourgram}":{"frequency":frequency_list,"fivegram":fivegram_list,"sixgram":sixgram_list,
        "sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list}}}
        fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("fourgram_data").document(f"fourgram{index}").set(dict_var)
except:
    print("Oops!", index, sys.exc_info()[0], "occurred.")       

##### Fivegram

In [19]:
df_fivegram

,index,fivegram,frequency,sixgram,sevengram,eightgram,ninegram,tengram
0,0,başka bir şey var mı,14104,"['istediğin başka bir şey var mı', 'yapabilece...","['için yapabileceğim başka bir şey var mı', 'b...",['sizin için yapabileceğim başka bir şey var m...,['sizin için yapabileceğim başka bir şey var m...,['affedersiniz bayan sizin için yapabileceğim ...
1,1,bu da ne demek oluyor,10205,"['bu da ne demek oluyor şimdi', 'bu da ne deme...","['ama bu da ne demek oluyor şimdi', 'alınmayın...",['alınmayın ama bu da ne demek oluyor şimdi'],[],['bu da ne demek oluyor yargılamak gibi bir şe...
2,2,o kadar da kötü değil,7012,"['bu o kadar da kötü değil', 'o kadar da kötü ...","['o kadar da kötü değil değil mi', 'hadi ama o...","['bu o kadar da kötü değil değil mi', 'ama o k...",['hadi ama pascal burası o kadar da kötü değil...,['hem prenses olmak o kadar da kötü değil deği...
3,3,sence de öyle değil mi,6305,"['görünüyor sence de öyle değil mi', 'var senc...","['iyi gidiyor sence de öyle değil mi', 'zamand...","['hoş bir vadi sence de öyle değil mi', 'çok d...","['burası hoş bir vadi sence de öyle değil mi',...",['gelecek geçmişten çok daha ilginç sence de ö...
4,4,sana bir şey sorabilir miyim,6224,"['hey sana bir şey sorabilir miyim', 'baba san...",['kimseye söylemezsen sana bir şey sorabilir m...,[],[],[]
...,...,...,...,...,...,...,...,...
87094,87094,beni çaldın ben de kendimi,19,"['beni çaldın ben de kendimi geri', 'sen beni ...","['beni çaldın ben de kendimi geri almaya', 'se...","['sen beni çaldın ben de kendimi geri almaya',...",['sen beni çaldın ben de kendimi geri almaya g...,[]
87095,87095,madem bir melek gibi çalıyorum,19,['madem bir melek gibi çalıyorum neden'],['madem bir melek gibi çalıyorum neden beni'],['madem bir melek gibi çalıyorum neden beni di...,[],[]
87096,87096,ile matrix arasında seçim yapmam,19,"['ile matrix arasında seçim yapmam gerekse', '...","['bu yaşam ile matrix arasında seçim yapmam', ...",['bu yaşam ile matrix arasında seçim yapmam ge...,[],[]
87097,87097,beklemek için nasıl bir yer,19,['beklemek için nasıl bir yer burası'],[],[],[],[]


In [20]:
df_fivegram["sixgram"] = df_fivegram["sixgram"].apply(eval)
df_fivegram["sevengram"] = df_fivegram["sevengram"].apply(eval)
df_fivegram["eightgram"] = df_fivegram["eightgram"].apply(eval)
df_fivegram["ninegram"] = df_fivegram["ninegram"].apply(eval)
df_fivegram["tengram"] = df_fivegram["tengram"].apply(eval)

In [21]:
index = df_fivegram["index"].to_list()
fivegram = df_fivegram["fivegram"].to_list()
frequency_list = df_fivegram["frequency"].to_list()
sixgram_list = df_fivegram["sixgram"].to_list()
sevengram_list = df_fivegram["sevengram"].to_list()
eightgram_list = df_fivegram["eightgram"].to_list()
ninegram_list = df_fivegram["ninegram"].to_list()
tengram_list = df_fivegram["tengram"].to_list()

In [22]:
firestore_fivegram_data = zip(index,fivegram,frequency_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [23]:
try:
    for index,fivegram,frequency_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_fivegram_data:
        dict_var = {f"{index}":{f"{fivegram}":{"frequency":frequency_list,"sixgram":sixgram_list,
        "sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list}}}
        #fivegram_dict.update(dict_var)
        fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("fivegram_data").document(f"fivegram{index}").set(dict_var)
except:
    print("Oops!", index, sys.exc_info()[0], "occurred.")     

In [ ]:
#fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("fivegram_data").document(f"details").set(fivegram_dict)

##### Sixgram

In [19]:
df_sixgram

,index,sixgram,frequency,sevengram,eightgram,ninegram,tengram
0,0,sana söylemem gereken bir şey var,3646,"['ama sana söylemem gereken bir şey var', 'önc...","['benim de sana söylemem gereken bir şey var',...",['sana söylemem gereken bir şey var ama nasıl ...,['sana söylemem gereken bir şey var ama nasıl ...
1,1,bir iki üç dört beş altı,2025,"['bir iki üç dört beş altı yedi', 've bir iki ...","['bir iki üç dört beş altı yedi sekiz', 've bi...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
2,2,söylemek istediğin bir şey var mı,1794,"['bana söylemek istediğin bir şey var mı', 'bi...",['önce bana söylemek istediğin bir şey var mı'...,['çıkmadan önce bana söylemek istediğin bir şe...,['arabadan çıkmadan önce bana söylemek istediğ...
3,3,daha iyi bir fikrin var mı,1792,"['senin daha iyi bir fikrin var mı', 'peki dah...","['peki daha iyi bir fikrin var mı şerif', 'lan...",[],['benim dahil olmadığım daha iyi bir fikrin va...
4,4,bunun iyi bir fikir olduğunu sanmıyorum,1743,['ama bunun iyi bir fikir olduğunu sanmıyorum'...,['hayır hayır bunun iyi bir fikir olduğunu san...,[],[]
...,...,...,...,...,...,...,...
199995,199995,iş yapmak için yeterli bilgiye sahip,10,"['bir iş yapmak için yeterli bilgiye sahip', '...",['bir iş yapmak için yeterli bilgiye sahip olm...,[],[]
199996,199996,söylediğin gibi çok cesurca bir hareketti,10,['getirmek söylediğin gibi çok cesurca bir har...,['oraya getirmek söylediğin gibi çok cesurca b...,['babamı oraya getirmek söylediğin gibi çok ce...,['gidip babamı oraya getirmek söylediğin gibi ...
199997,199997,kutunun içinde hafif bir negatif basınç,10,['kutunun içinde hafif bir negatif basınç var'],[],[],[]
199998,199998,gidebilecek yaşa geldiğinde hiçbir havası kalm...,10,['gidebilecek yaşa geldiğinde hiçbir havası ka...,['ona gidebilecek yaşa geldiğinde hiçbir havas...,[],[]


In [20]:
df_sixgram["sevengram"] = df_sixgram["sevengram"].apply(eval)
df_sixgram["eightgram"] = df_sixgram["eightgram"].apply(eval)
df_sixgram["ninegram"] = df_sixgram["ninegram"].apply(eval)
df_sixgram["tengram"] = df_sixgram["tengram"].apply(eval)

In [21]:
index = df_sixgram["index"].to_list()
sixgram = df_sixgram["sixgram"].to_list()
frequency_list = df_sixgram["frequency"].to_list()
sevengram_list = df_sixgram["sevengram"].to_list()
eightgram_list = df_sixgram["eightgram"].to_list()
ninegram_list = df_sixgram["ninegram"].to_list()
tengram_list = df_sixgram["tengram"].to_list()

In [22]:
firestore_sixgram_data = zip(index,sixgram,frequency_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [23]:
try:
    for index,sixgram,frequency_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_sixgram_data:
        dict_var = {f"{index}":{f"{sixgram}":{"frequency":frequency_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list}}}
        #sixgram_dict.update(dict_var)   
        fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("sixgram_data").document(f"sixgram{index}").set(dict_var)
except:
    print("Oops!", index, sys.exc_info()[0], "occurred.")    

In [ ]:
#fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("sixgram_data").document(f"details").set(sixgram_dict)

##### Sevengram

In [13]:
df_sevengram

,index,sevengram,frequency,eightgram,ninegram,tengram
0,0,bir iki üç dört beş altı yedi,1455,"['bir iki üç dört beş altı yedi sekiz', 've bi...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,1,iki üç dört beş altı yedi sekiz,1080,"['bir iki üç dört beş altı yedi sekiz', 'iki ü...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
2,2,bana söylemek istediğin bir şey var mı,752,['önce bana söylemek istediğin bir şey var mı'...,['çıkmadan önce bana söylemek istediğin bir şe...,['arabadan çıkmadan önce bana söylemek istediğ...
3,3,yedi altı beş dört üç iki bir,596,"['sekiz yedi altı beş dört üç iki bir', 'yedi ...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
4,4,sekiz yedi altı beş dört üç iki,551,"['dokuz sekiz yedi altı beş dört üç iki', 'sek...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
...,...,...,...,...,...,...
102080,102080,akşam üçüncü ve muhtemelen son olarak yerde,9,['bu akşam üçüncü ve muhtemelen son olarak yer...,['atom bu akşam üçüncü ve muhtemelen son olara...,[]
102081,102081,herifi durdurmak için sadece bir saat kırk,9,['herifi durdurmak için sadece bir saat kırk d...,['bu herifi durdurmak için sadece bir saat kır...,['bu herifi durdurmak için sadece bir saat kır...
102082,102082,görücü usulü evlenmek daha çok işe yarıyor,9,['bazen görücü usulü evlenmek daha çok işe yar...,[],[]
102083,102083,konuda çok yanlış olduğundan kesinlikle eminim...,9,['bu konuda çok yanlış olduğundan kesinlikle e...,[],[]


In [14]:
df_sevengram["eightgram"] = df_sevengram["eightgram"].apply(eval)
df_sevengram["ninegram"] = df_sevengram["ninegram"].apply(eval)
df_sevengram["tengram"] = df_sevengram["tengram"].apply(eval)

In [15]:
index = df_sevengram["index"].to_list()
sevengram = df_sevengram["sevengram"].to_list()
frequency_list = df_sevengram["frequency"].to_list()
eightgram_list = df_sevengram["eightgram"].to_list()
ninegram_list = df_sevengram["ninegram"].to_list()
tengram_list = df_sevengram["tengram"].to_list()

In [16]:
firestore_sevengram_data = zip(index,sevengram,frequency_list,eightgram_list,ninegram_list,tengram_list)

In [17]:
try:
    for index,sevengram,frequency_list,eightgram_list,ninegram_list,tengram_list in firestore_sevengram_data:
        dict_var = {f"{index}":{f"{sevengram}":{"frequency":frequency_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list}}}
        #sevengram_dict.update(dict_var)  
        fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("sevengram_data").document(f"sevengram{index}").set(dict_var)
except:
    print("Oops!", index, sys.exc_info()[0], "occurred.")    

In [ ]:
#fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("sevengram_data").document(f"details").set(sevengram_dict)

##### Eightgram

In [13]:
df_eightgram

,index,eightgram,frequency,ninegram,tengram
0,0,bir iki üç dört beş altı yedi sekiz,1060,"['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,1,dokuz sekiz yedi altı beş dört üç iki,513,"['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
2,2,sekiz yedi altı beş dört üç iki bir,497,"['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
3,3,iki üç dört beş altı yedi sekiz dokuz,411,"['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
4,4,üç dört beş altı yedi sekiz dokuz on,329,"['iki üç dört beş altı yedi sekiz dokuz on', '...",['bir iki üç dört beş altı yedi sekiz dokuz on...
...,...,...,...,...,...
71092,71092,bu bok en uzun süre ne kadar etki,7,[],['bu bok en uzun süre ne kadar etki gösterdi s...
71093,71093,şu anda telefona gelemez ama beni bıraktığınız...,7,[],['joshua şu anda telefona gelemez ama beni bır...
71094,71094,istediğim elini içeri sokman ve kabloyu nazikç...,7,[],['senden istediğim elini içeri sokman ve kablo...
71095,71095,insan diğer bir insanın işine çomak sokmak zor...,7,[],['neden bir insan diğer bir insanın işine çoma...


In [14]:
df_eightgram["ninegram"] = df_eightgram["ninegram"].apply(eval)
df_eightgram["tengram"] = df_eightgram["tengram"].apply(eval)

In [15]:
index = df_eightgram["index"].to_list()
eightgram = df_eightgram["eightgram"].to_list()
frequency_list = df_eightgram["frequency"].to_list()
ninegram_list = df_eightgram["ninegram"].to_list()
tengram_list = df_eightgram["tengram"].to_list()

In [16]:
firestore_eightgram_data = zip(index,eightgram,frequency_list,ninegram_list,tengram_list)

In [17]:
try:
    for index,eightgram,frequency_list,ninegram_list,tengram_list in firestore_eightgram_data:
        dict_var = {f"{index}":{f"{eightgram}":{"frequency":frequency_list,"ninegram":ninegram_list,"tengram":tengram_list}}}
        #eightgram_dict.update(dict_var)  
        fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("eight_data").document(f"eight{index}").set(dict_var)
except:
    print("Oops!", index, sys.exc_info()[0], "occurred.")    

In [ ]:
#fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("eightgram_data").document(f"details").set(eightgram_dict)

##### Ninegram

In [12]:
df_ninegram

,index,ninegram,frequency,tengram
0,0,dokuz sekiz yedi altı beş dört üç iki bir,463,['on dokuz sekiz yedi altı beş dört üç iki bir...
1,1,bir iki üç dört beş altı yedi sekiz dokuz,405,['bir iki üç dört beş altı yedi sekiz dokuz on...
2,2,iki üç dört beş altı yedi sekiz dokuz on,287,['bir iki üç dört beş altı yedi sekiz dokuz on']
3,3,on dokuz sekiz yedi altı beş dört üç iki,277,['on dokuz sekiz yedi altı beş dört üç iki bir']
4,4,i t a r i s y e n,128,['g i t a r i s y e n']
...,...,...,...,...
20237,20237,sevgili kızım aşık bir kadın için bile mantıks...,7,['sevgili kızım aşık bir kadın için bile mantı...
20238,20238,çene çalmayı bir yana bırakıp bu işi gerçekten...,7,['çene çalmayı bir yana bırakıp bu işi gerçekt...
20239,20239,merak etme böyle bir şey yapmamı istemediğinde...,7,['merak etme böyle bir şey yapmamı istemediğin...
20240,20240,sabah dokuz gibi geçerken uğrarım bu yüzden ön...,7,['sabah dokuz gibi geçerken uğrarım bu yüzden ...


In [13]:
df_ninegram["tengram"] = df_ninegram["tengram"].apply(eval)

In [14]:
index = df_ninegram["index"].to_list()
ninegram = df_ninegram["ninegram"].to_list()
frequency_list = df_ninegram["frequency"].to_list()
tengram_list = df_ninegram["tengram"].to_list()

In [15]:
firestore_ninegram_data = zip(index,ninegram,frequency_list,tengram_list)

In [16]:
try:
    for index,ninegram,frequency_list,tengram_list in firestore_ninegram_data:
        dict_var = {f"{index}":{f"{ninegram}":{"frequency":frequency_list,"tengram":tengram_list}}}
        #ninegram_dict.update(dict_var)
        fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("ninegram_data").document(f"ninegram{index}").set(dict_var)
except:
    print("Oops!", index, sys.exc_info()[0], "occurred.")  

In [ ]:
#fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("ninegram_data").document(f"details").set(ninegram_dict)

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass